# <center>Выпускной проект, часть 2: SQL</center>

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<img src="../pict/chapter_13/data_schema.png">

## Импорт и подключение к БД

In [1]:
# !pip install psycopg2-binary

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
def query(query):
    '''Функция возвращает рещультат запроса к БД'''
    return pd.io.sql.read_sql(query, con = engine)

Проверю, что в базе данных есть все необходимые мне таблицы из условия задания. 

In [5]:
query('''
SELECT 
    table_name 
FROM 
    information_schema.tables
WHERE 
    table_schema NOT IN ('information_schema', 'pg_catalog')
    AND table_name IN('books', 'authors', 'ratings', 'reviews', 'publishers')
''')

,table_name
0,publishers
1,authors
2,reviews
3,ratings
4,books


Я не стал усложнять и делать цикл/функцию которая бы перебирала все нужные мне имена таблиц и выводила бы в отельный вывод типа данных и названия столбцов. Решил сделать проверку силами самой БД.

In [6]:
z = 'books'
query('''
SELECT 
    table_name, column_name, data_type 
FROM 
    INFORMATION_SCHEMA.COLUMNS 
WHERE 
    table_name IN ('books', 'authors', 'ratings', 'reviews', 'publishers')
ORDER BY
    table_name
''')

,table_name,column_name,data_type
0,authors,author_id,integer
1,authors,author,text
2,books,publisher_id,integer
3,books,publication_date,date
4,books,num_pages,integer
5,books,author_id,integer
6,books,title,text
7,books,book_id,integer
8,publishers,publisher_id,integer
9,publishers,publisher,text


Судя по всему с данными все в порядке. Все необходимые таблицы присутствуют в базе данных. Типы данных тоже не вызывают вопросов.


## Посчитайте количество книг, выпущенных после 1 января 2000 года;

In [7]:
query('''
SELECT 
    COUNT(book_id) AS count_book
FROM 
    books
WHERE 
    publication_date > '2000-01-01'
''')

,count_book
0,819


После 1-го января 2000 года было выпущено 819 книг 

## Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги;

In [9]:
query('''
SELECT
    books.book_id,
    books.title,
    COUNT(DISTINCT reviews.review_id) AS cnt_user_review,
    AVG(ratings.rating) AS avg_rating
FROM
    books
INNER JOIN ratings ON books.book_id = ratings.book_id
INNER JOIN reviews ON books.book_id = reviews.book_id
GROUP BY
    books.book_id
ORDER BY
    cnt_user_review DESC
''')

,book_id,title,cnt_user_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,854,The Road,6,3.772727
2,656,The Book Thief,6,4.264151
3,734,The Glass Castle,6,4.206897
4,963,Water for Elephants,6,3.977273
...,...,...,...,...
989,465,Naked Empire (Sword of Truth #8),1,3.500000
990,446,Moo Baa La La La!,1,3.000000
991,431,Merrick (The Vampire Chronicles #7),1,4.000000
992,92,Babyville,1,3.500000


## Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры);

In [10]:
query('''
SELECT
    publishers.publisher,
    COUNT(books.book_id) AS count_books
FROM 
    books
INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    publishers.publisher
ORDER BY
    count_books DESC
''')

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Определил. Получается, что лидер по изданию количества книг больше 50 страниц это - `Penguin Books`

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;


In [12]:
query('''
SELECT 
    authors.author,
    AVG(ratings.rating) AS avg_rating
FROM 
    books
LEFT JOIN ratings on books.book_id = ratings.book_id
LEFT JOIN authors on books.author_id = authors.author_id
WHERE
    books.book_id IN (
        SELECT ratings.book_id
        FROM ratings
        GROUP BY ratings.book_id
        HAVING COUNT(ratings.rating) > 50)
GROUP BY
    authors.author
ORDER BY
    avg_rating DESC
''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


Самая высокая средняя оценка книг у автора `J.K. Rowling/Mary GrandPré`

## Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [15]:
query('''
SELECT
    ROUND(COUNT(review_id) / COUNT(DISTINCT username) :: decimal,1) AS result
FROM
    reviews
WHERE
    -- Делаю вложенный запрос, чтобы отобрать пользователей с количеством оценок > 50 
    reviews.username IN 
    (SELECT 
        username
    FROM 
        ratings
    GROUP BY
        username
    HAVING 
        COUNT(rating) > 50) 
''')

,result
0,24.3
